Here, we will do NLP with 2 methods

1)Use Word2Vec to create word embeddings and then pass onto RNN

2)Use inbuilt embedding layer of keras

In [7]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('cleaned_sentiment.csv')
df.head()

,target,comment_text
0,0.166667,What a breathe of fresh air to have someone wh...
1,0.600000,Your jewish friends were the ones who told you...
2,0.000000,Possible collusion by Trump and his affiliates...
3,0.000000,Exactly. We need a % of GDP spending cap at t...
4,0.000000,"By your own comment, even if some of them vote..."


In [3]:
df[df['comment_text'].isnull()]

,target,comment_text
10108,0.0,NaN


In [4]:
df.drop(10108,inplace=True)

In [7]:
from nltk.corpus import stopwords
sw=stopwords.words('english')

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range(0,len(df)):
    sentenceTemp=(df.iloc[i,1])
   
    sentenceTemp=sentenceTemp.lower()
    sentenceTemp=sentenceTemp.split()
    sentenceTemp=[re.sub(r'[^\w\s]', '', word)  for word in sentenceTemp]
    sentenceTemp=[re.sub(r'[^a-zA-Z0-9]', '', word)  for word in sentenceTemp]
    sentenceTemp=[lemmatizer.lemmatize(word) for word in sentenceTemp if word not in sw]
    sentenceTemp=" ".join(sentenceTemp)
    corpus.append(sentenceTemp)

In [9]:
y=df['target']

In [10]:
import gensim.downloader as api
wv = api.load('glove-twitter-100')


[==================================================] 100.0% 387.1/387.1MB downloaded


In [11]:
x=[]
for i in corpus:
    sentence=i
    words=sentence.split()
    temp=[]
    for word in words:
        if word in wv:
            temp.append(wv[word])
        else:
            temp.append(np.zeros(100))
    
    x.append(temp)
    

In [13]:
(x[0]) # so each sentence is represented as (number of words*300) (here we have not taken avg word2vec)

[array([-0.67515  ,  0.57084  ,  0.46594  , -0.22116  , -0.63343  ,
         0.14801  ,  0.97672  , -0.021585 , -0.30904  ,  0.20697  ,
         0.11791  , -0.028849 , -3.0109   , -0.031893 , -1.0401   ,
        -0.044973 ,  0.038306 , -0.086969 ,  0.83777  , -0.30544  ,
        -0.98768  ,  0.091357 ,  0.70263  ,  0.53273  ,  0.67068  ,
         0.82231  ,  0.18931  , -0.05535  , -0.20008  ,  0.36914  ,
        -1.0221   , -0.31693  , -0.55308  , -0.012306 , -1.1186   ,
        -0.2057   , -0.087288 , -0.17575  , -0.15271  ,  0.13768  ,
        -0.48892  , -0.36096  ,  0.028857 ,  0.21561  ,  0.13633  ,
        -0.022644 ,  1.1417   ,  0.41053  , -0.14174  , -0.18602  ,
         0.25032  ,  0.66865  ,  0.76561  , -0.20742  ,  0.34512  ,
        -0.48538  ,  0.13678  ,  0.48904  ,  0.40368  ,  0.2624   ,
         0.37286  ,  0.25062  , -0.18111  , -0.38976  ,  0.48985  ,
        -0.49186  , -0.14739  ,  0.62565  , -0.16332  ,  0.67144  ,
        -0.013243 ,  0.98089  ,  0.44725  , -0.9

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_padded=pad_sequences(x,dtype='float32', padding='post', maxlen=30)

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_padded, y, test_size=0.2, random_state=42)


In [26]:
print(x_train.shape)


(79999, 30, 100)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

model = Sequential()
model.add(SimpleRNN(64, input_shape=(30, 100), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # regression output
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 64)                10560     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________


In [29]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=32
)


Epoch 1/10
2500/2500 [==============================] - 44s 16ms/step - loss: 0.0532 - mae: 0.1559 - val_loss: 0.0318 - val_mae: 0.1293
Epoch 2/10
2500/2500 [==============================] - 41s 16ms/step - loss: 0.0359 - mae: 0.1345 - val_loss: 0.0357 - val_mae: 0.1341
Epoch 3/10
2500/2500 [==============================] - 40s 16ms/step - loss: 0.0371 - mae: 0.1369 - val_loss: 0.0348 - val_mae: 0.1344
Epoch 4/10
2500/2500 [==============================] - 44s 17ms/step - loss: 0.0381 - mae: 0.1395 - val_loss: 0.0367 - val_mae: 0.1390
Epoch 5/10
2500/2500 [==============================] - 41s 17ms/step - loss: 0.0341 - mae: 0.1281 - val_loss: 0.0327 - val_mae: 0.1362
Epoch 6/10
2500/2500 [==============================] - 43s 17ms/step - loss: 0.0323 - mae: 0.1247 - val_loss: 0.0374 - val_mae: 0.1334
Epoch 7/10
2500/2500 [==============================] - 41s 17ms/step - loss: 0.0381 - mae: 0.1394 - val_loss: 0.0391 - val_mae: 0.1615
Epoch 8/10
2500/2500 [==========================

In [19]:
import psutil
print(psutil.virtual_memory())


svmem(total=16869548032, available=791396352, percent=95.3, used=16078151680, free=791396352)


In [20]:
import gc

del wv   # remove the pretrained model from memory
gc.collect()  # force garbage collection


1465

In [21]:
import psutil
print(psutil.virtual_memory())


svmem(total=16869548032, available=2880049152, percent=82.9, used=13989498880, free=2880049152)


In [23]:
import psutil
print(psutil.virtual_memory())


svmem(total=16869548032, available=2805960704, percent=83.4, used=14063587328, free=2805960704)


In [30]:
from sklearn.metrics import r2_score 
y_pred=model.predict(x_test) 
print(r2_score(y_test,y_pred))

625/625 [==============================] - 3s 4ms/step
0.1959107518196106


In [33]:
model.save('sentimentAnalysis.h5')

In [32]:
import re

In [ ]:
import gensim.downloader as api
wv = api.load('glove-twitter-100')

def predict(sentence):
    sentence=sentence.lower()
    sentence=sentence.split()
    sentence=[re.sub(r'[^\w\s]', '', word)  for word in sentence]
    sentence=[re.sub(r'[^a-zA-Z0-9]', '', word)  for word in sentence]
    sentence=[lemmatizer.lemmatize(word) for word in sentence if word not in sw]
    sentence=" ".join(sentence)
    words=sentence.split()
    temp=[]
    for word in words:
        if word in wv:
            temp.append(wv[word])
        else:
            temp.append(np.zeros(100))
    temp=pad_sequences([temp],dtype='float32', padding='post', maxlen=30)
    score=model.predict(temp)
    return score
    

In [36]:
predict("This is the worst thing ever.")[0][0]


1/1 [==============================] - 0s 22ms/step


0.26209775

In [38]:
predict("You are absolutely horrible at this!")[0][0]



1/1 [==============================] - 0s 20ms/step


0.34964532

In [39]:
predict("You are an amazing person!")[0][0]

1/1 [==============================] - 0s 24ms/step


0.081078075

In [40]:
predict("This movie was okay, nothing special.")[0][0]

1/1 [==============================] - 0s 27ms/step


0.026401177

In [41]:
predict("That was such a stupid comment.")[0][0]

1/1 [==============================] - 0s 29ms/step


0.3496489

In [42]:
predict("I appreciate all your help and kindness.")[0][0]

1/1 [==============================] - 0s 21ms/step


-0.0028122216

In [43]:
predict("I really loved your work, it’s brilliant!")[0][0]

1/1 [==============================] - 0s 20ms/step


0.07873299

In [44]:
predict("This project is going great so far.")[0][0]

1/1 [==============================] - 0s 23ms/step


0.010052636

In [45]:
predict("You are such an idiot, I can’t believe how stupid you are.")[0][0]


1/1 [==============================] - 0s 29ms/step


0.3496437

In [46]:
predict("You are an amazing person, full of kindness and positivity. The world needs more people like you!")[0][0]


1/1 [==============================] - 0s 25ms/step


0.18222973

In [47]:
predict("You are the most disgusting and worthless person I’ve ever met. I hate everything about you.")[0][0]


1/1 [==============================] - 0s 32ms/step


0.22332717

Using inbuilt Embedding layer of keras:

In [8]:
df=pd.read_csv('cleaned_sentiment.csv')
df.head()

,target,comment_text
0,0.166667,What a breathe of fresh air to have someone wh...
1,0.600000,Your jewish friends were the ones who told you...
2,0.000000,Possible collusion by Trump and his affiliates...
3,0.000000,Exactly. We need a % of GDP spending cap at t...
4,0.000000,"By your own comment, even if some of them vote..."


In [9]:
df[df['comment_text'].isnull()]

,target,comment_text
10108,0.0,NaN


In [10]:
df.drop(10108,inplace=True)

In [11]:
from nltk.corpus import stopwords
sw=stopwords.words('english')

In [12]:
import re
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range(0,len(df)):
    sentenceTemp=(df.iloc[i,1])
   
    sentenceTemp=sentenceTemp.lower()
    sentenceTemp=sentenceTemp.split()
    sentenceTemp=[re.sub(r'[^\w\s]', '', word)  for word in sentenceTemp]
    sentenceTemp=[re.sub(r'[^a-zA-Z0-9]', '', word)  for word in sentenceTemp]
    sentenceTemp=[lemmatizer.lemmatize(word) for word in sentenceTemp if word not in sw]
    sentenceTemp=" ".join(sentenceTemp)
    corpus.append(sentenceTemp)

In [13]:
len(corpus)

99999

In [14]:
y=df['target']

In [25]:
corpus

['breathe fresh air someone embrace common sense value instead leadership canada clearly different page perhaps reading different book',
 'jewish friend one told zionist control canada believable hahaha jewish friend get info white supremacist web site publish site given jew control medium theyre taking time controlling bank',
 'possible collusion trump affiliate debunked state matter one focus ongoing investigation existence ongoing investigation collusion factual even though might conflict diatribe political spin person echo chamber',
 'exactly need  gdp spending cap federal level 18 wouldnt mind paying tax much meant paying debt instead spend every dime anytime get hand money',
 'comment even voted ndp pq trudeau demonstrably less partisan appointment course ndp dont support senate actually put money mouth',
 'thought care much put money mouth buy thing',
 'kboo  born vetting go immigrant applicant vetted',
 'keep degenerate jail life dont need registeredproblem solved maybe blaming

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

x=[]
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
x=tokenizer.texts_to_sequences(corpus)

In [16]:
x

[[7562,
  3015,
  701,
  100,
  3355,
  455,
  309,
  305,
  222,
  638,
  36,
  438,
  216,
  1218,
  238,
  551,
  216,
  533],
 [2091,
  314,
  3,
  398,
  10420,
  237,
  36,
  8305,
  5102,
  2091,
  314,
  6,
  1839,
  137,
  2043,
  3308,
  666,
  3409,
  666,
  270,
  1563,
  237,
  173,
  363,
  333,
  10,
  3898,
  556],
 [466,
  2145,
  5,
  15169,
  5168,
  16,
  169,
  3,
  945,
  2713,
  631,
  2382,
  2713,
  631,
  2145,
  3087,
  19,
  260,
  124,
  1147,
  7062,
  139,
  1977,
  109,
  5605,
  3899],
 [325,
  14,
  2332,
  458,
  2316,
  211,
  249,
  1659,
  385,
  283,
  342,
  20,
  28,
  1091,
  342,
  529,
  222,
  541,
  61,
  3023,
  3572,
  6,
  242,
  31],
 [57,
  19,
  423,
  848,
  14418,
  215,
  9230,
  146,
  1846,
  2522,
  176,
  848,
  7,
  96,
  694,
  82,
  116,
  31,
  1302],
 [197, 75, 28, 116, 31, 1302, 316, 29],
 [42094, 844, 3657, 23, 516, 5677, 5453],
 [85,
  10732,
  902,
  53,
  7,
  14,
  42095,
  3123,
  110,
  1994,
  550,
  2,
  14419,
 

In [17]:
len(x[0])

18

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x=pad_sequences(x,padding="post")
x

array([[7562, 3015,  701, ...,    0,    0,    0],
       [2091,  314,    3, ...,    0,    0,    0],
       [ 466, 2145,    5, ...,    0,    0,    0],
       ...,
       [ 115,  100, 2705, ...,    0,    0,    0],
       [1411,  353, 2979, ...,    0,    0,    0],
       [  82,   15,  106, ...,    0,    0,    0]])

In [20]:
len(x[0])

132

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [22]:
x_train.shape

(79999, 132)

In [23]:
x_train.shape[1]

132

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding,Dropout
vocab_size = len(tokenizer.word_index) + 1
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=16,input_length=132))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # regression output
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 132, 16)           1541344   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                5184      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,546,593
Trainable params: 1,546,593
Non-trainable params: 0
_________________________________________________________________


In [40]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=32
)
 

Epoch 1/10
2500/2500 [==============================] - 255s 101ms/step - loss: 0.0390 - mae: 0.1439 - val_loss: 0.0386 - val_mae: 0.1426
Epoch 2/10
2500/2500 [==============================] - 253s 101ms/step - loss: 0.0387 - mae: 0.1439 - val_loss: 0.0386 - val_mae: 0.1475
Epoch 3/10
2500/2500 [==============================] - 422s 169ms/step - loss: 0.0387 - mae: 0.1441 - val_loss: 0.0386 - val_mae: 0.1449
Epoch 4/10
2500/2500 [==============================] - 558s 223ms/step - loss: 0.0387 - mae: 0.1441 - val_loss: 0.0386 - val_mae: 0.1439
Epoch 5/10
2500/2500 [==============================] - 556s 223ms/step - loss: 0.0386 - mae: 0.1441 - val_loss: 0.0386 - val_mae: 0.1443
Epoch 6/10
2500/2500 [==============================] - 699s 280ms/step - loss: 0.0386 - mae: 0.1441 - val_loss: 0.0386 - val_mae: 0.1453
Epoch 7/10
2500/2500 [==============================] - 611s 244ms/step - loss: 0.0399 - mae: 0.1446 - val_loss: 0.0395 - val_mae: 0.1419
Epoch 8/10
2500/2500 [============

In [ ]:
model.save('sentimentAnalysis2.h5')

In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
model=load_model('sentimentAnalysis2.h5')

In [24]:
from sklearn.metrics import r2_score 
y_pred=model.predict(x_test) 
print(r2_score(y_test,y_pred))

625/625 [==============================] - 10s 12ms/step
-0.06130410058787494


In [45]:
def predict2(sentence):
    sentence=sentence.lower()
    sentence=sentence.split()
    sentence=[re.sub(r'[^\w\s]', '', word)  for word in sentence]
    sentence=[re.sub(r'[^a-zA-Z0-9]', '', word)  for word in sentence]
    sentence=[lemmatizer.lemmatize(word) for word in sentence if word not in sw]
    sentence=" ".join(sentence)
    inputSentence=tokenizer.texts_to_sequences([sentence])
    inputSentence=pad_sequences(inputSentence,padding='post',maxlen=132)
    senti_score=model.predict(inputSentence)
    return senti_score



In [47]:
predict2("This is absolutely wonderful, I’m so proud of you!")

1/1 [==============================] - 0s 53ms/step


array([[0.07980284]], dtype=float32)

In [48]:
predict2("You are the worst person I have ever met")

1/1 [==============================] - 0s 55ms/step


array([[0.09248494]], dtype=float32)

In [49]:
predict2("Everything about this project is stupid and pointless")

1/1 [==============================] - 0s 46ms/step


array([[0.03855295]], dtype=float32)

In [51]:
predict2("I’m disappointed with the results, I expected more.")

1/1 [==============================] - 0s 52ms/step


array([[0.03790776]], dtype=float32)